In [1]:
import pandas as pd
import glob
import nltk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# import spacy
from textblob import TextBlob


import requests
from datetime import timedelta, date
from bs4 import BeautifulSoup


In [2]:
def options(option,data,types):
    if(option=="vader"):
        vaderAnalysis(data,types)
        
    if(option=="nltk"):
        nltkClassifier(data,types)
        
    if(option=="both"):
        vaderAnalysis(data,types)
#         print("\n")
        nltkClassifier(data,types)

In [3]:
def vaderAnalysis(data,types):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

    analyzer = SentimentIntensityAnalyzer()
    pos = 0
    neg = 0

    vs = analyzer.polarity_scores(data)
    pos = pos+vs['pos']
    neg = neg+vs['neg']
    
    polarity = round(pos-neg,3)
    if(pos>neg):
        print("VADER -",types,", Positive News = {}".format(polarity))
    else:
        print("VADER -",types,", Negative News = {}".format(polarity))
#     print("Positivity - {}, Negativity - {}, Polarity - {}".format(pos, neg,polarity))
#     print("VADER -",types,", = {}".format(polarity))


In [4]:
def nltkClassifier(data,types):
    
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sid = SentimentIntensityAnalyzer()
    pos_word_list = []

    neu_word_list = []
    neg_word_list = []

    overall = 0
    l=data
    blob = TextBlob(l)
    l = l.lower()
    for word in l.split():
        print(sid.polarity_scores(word)['compound'], word)
        overall = overall + sid.polarity_scores(word)['compound']
        if (sid.polarity_scores(word)['compound']) >= 0.02:
            pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.02:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)
    overall = round(overall,3)
    if(overall < -0.08):
        print("NLTK  -",types,", Negative news = ", overall)
    elif(overall > 0.08):
        print("NLTK  -",types,", Positive news = ", overall)
    else:
        print("NLTK  -",types,", Neutral  news = ", overall)

In [5]:
def downloadUrl(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    title = soup.find('h1', {"id": "page-title"}).get_text()

    contents = soup.find_all('div', {"class": "content"})
    data = ""
    for content in contents:
        data = data + content.get_text().replace("\n", "").replace("\"", "")

    return (title,data)

In [6]:

def downloadMtlUrl(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    title = soup.find('h1', {"class": "artTitle"}).get_text()
    data = ""

    sp = soup.find('div', {"id": "article-main"})
    for p in sp.findAll('p'):
        o = p.get_text()
        if(not o.startswith("Disclaimer")):
            data = data + o.replace("\n", "").replace("\"", "")

    return(title, data)

In [7]:
def downloadMtlAll(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    i = 0
    b = []
    a = []
    for title in soup.findAll('h2'):
        b.append('')
        a.append('')
        b[i] = title.get_text()
        for z in title.findAll('a'):
            a[i] = z.get('href')
        i = i+1

    return(a)


In [8]:
from datetime import timedelta, date
import re
import requests
url = "https://www.deccanherald.com/getarchive"


def normalizeSpaces(input):
    return input.strip().replace(" ", "-").lower()


def normalizeSentence(input):
    return re.sub(r'[\'\,\:\/\\\(\)]', '', input.strip().lower())

def printLinks(currentDate):
    outs=[]
    response = requests.post(url, json={'arcDate': currentDate})
    soup = BeautifulSoup(response.content, "html.parser")

    headlines = soup.find_all('h2')
    contents = soup.find_all('ul', {'class': 'group'})

    for idx, content in enumerate(contents):
        contentLinks = content.findChildren('li')
        for contentLink in contentLinks:
            hL = headlines[idx].string
            if(hL.startswith("Business")):
                outs.append("https://www.deccanherald.com"+str(contentLink.a['href']))
                
    return outs

urls = printLinks("2020/02/11")
print("Done")

Done


In [9]:
# urls = ["https://www.deccanherald.com/business/amazon-india-inks-long-term-deals-with-future-retail-791766.html","https://www.deccanherald.com/business/business-news/sensex-tanks-788-pts-nifty-falls-235-pts-791748.html","https://www.deccanherald.com/business/business-news/india-incs-overseas-borrowing-up-65-in-nov-2019-791533.html"]
# urls=["https://www.moneycontrol.com/news/business/tech-mahindra-q3-pat-may-dip-10-3-qoq-to-rs-1008-cr-motilal-oswal-4792611.html"]


#For deccan herald


#For moneycontrol
# urls=downloadMtlAll("https://www.moneycontrol.com/news/news-all/page-1/")
urls=downloadMtlAll("https://www.moneycontrol.com/news/business/markets/")

for url in urls:
    if(url!=''):
        title,data = downloadMtlUrl(url)
#         title,data = downloadUrl(url)
        print("\n------\nHeadline - ", title,"\n")
#         options("both",title,"Headline")
#         print("\n=\n")
        options("both",data,"Full")
        print(url)


------
Headline -  Mahanagar Gas Q3 operating earnings weak; key highlights from analysts concall 

VADER - Full , Positive News = 0.011
NLTK  - Full , Positive news =  0.697
https://www.moneycontrol.com/news/business/earnings/mahanagar-gas-q3-operating-earnings-weak-key-highlights-from-analysts-concall-4933001.html

------
Headline -  Berger Paints Q3 profit boosted by operating income; key highlights from analysts concall 

VADER - Full , Positive News = 0.057
NLTK  - Full , Positive news =  5.298
https://www.moneycontrol.com/news/business/earnings/berger-paints-q3-profit-boosted-by-operating-income-key-highlights-from-analysts-concall-4933121.html

------
Headline -  Trade Setup for Wednesday: Top 15 things to know before Opening Bell 

VADER - Full , Positive News = 0.105
NLTK  - Full , Positive news =  12.674
https://www.moneycontrol.com/news/business/markets/trade-setup-for-wednesday-top-15-things-to-know-before-opening-bell-81-4933551.html

------
Headline -  IDBI Bank Q3 loss 